In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.26.0 to work with mlw_sdk


In [ ]:
if False:
    from azureml.core import Datastore, Dataset
    import pandas as pd
    import os

    # Set default data store
    #ws.set_default_datastore('workspaceblobstore')
    default_ds = ws.get_default_datastore()

    # Enumerate all datastores, indicating which is the default
    for ds_name in ws.datastores:
        print(ds_name, "- Default =", ds_name == default_ds.name)

    # Load the diabetes data
    test = Dataset.get_by_id(ws, '0bc4c13e-638b-48ad-a070-29bfc88faee0').to_pandas_dataframe()
    
    # Get a 100-item sample of the feature columns (not the diabetic label)
    

    # Create a folder
    batch_folder = './batch-data'
    os.makedirs(batch_folder, exist_ok=True)
    print("Folder created!")

    # Save each sample as a separate file
    print("Saving files...")
    for i in range(100):
        fname = str(i+1) + '.csv'
        sample[i].tofile(os.path.join(batch_folder, fname), sep=",")
    print("files saved!")

    # Upload the files to the default datastore
    print("Uploading files to datastore...")
    default_ds = ws.get_default_datastore()
    default_ds.upload(src_dir="batch-data", target_path="batch-data", overwrite=True, show_progress=True)

    # Register a dataset for the input data
    batch_data_set = Dataset.File.from_files(path=(default_ds, 'batch-data/'), validate=False)
    try:
        batch_data_set = batch_data_set.register(workspace=ws, 
                                                 name='batch-data',
                                                 description='batch data',
                                                 create_new_version=True)
    except Exception as ex:
        print(ex)

    print("Done!")

In [ ]:
%%writefile $batch_lin_reg.py
import os
import numpy as np
from azureml.core import Model
import joblib


def init():
    # Runs when the pipeline step is initialized
    global model

    # load the model
    model_path = Model.get_model_path('linear_reg.pkl')
    model = joblib.load(model_path)


def run(mini_batch):
    # This runs for each batch
    resultList = []

    # process each file in the batch
    for f in mini_batch:
        # Read the comma-delimited data into an array
        data = np.genfromtxt(f, delimiter=',')
        # Reshape into a 2-dimensional array for prediction (model expects multiple items)
        prediction = model.predict(data.reshape(1, -1))
        # Append prediction to results
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList